In [1]:
#! /usr/bin/python
# -*- coding: utf-8 -*-
# @author izhangxm
# Copyright 2017 izhangxm@gmail.com. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [21]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from itertools import product
import os.path as osp
from scipy.optimize import leastsq
import time
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
import math
import sunode
import sunode.wrappers.as_pytensor
from copy import deepcopy

# from tqdm import tqdm, trange
from tqdm.notebook import  tqdm
from IPython.display import display as print
from datetime import datetime

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

In [109]:
class MyDataset(object):

    def __init__(self, dataset_path):

        self.dataset_path = dataset_path
        
        df = pd.read_csv(dataset_path)
        
        self.df = df
        self._setup()
    
    def _setup(self):
        
        df = self.df
        # 数据初步处理
        # 计算反应速率rate，初始速率固定设置为0
        for c_i, (col_name, col_sr) in enumerate(df.items()):
            if "error" in col_name or "time" in col_name or "rate" in col_name:
                continue
            rate_col_name = f"{col_name}_rate"
            rates = []
            pre_t = None
            pre_v = None
            for th, (index, value) in zip(df['time'], col_sr.items()):
                if int(index) == 0:
                    rates.append(0.0)
                    pre_t = th
                    pre_v = value
                    continue

                delta_t = th - pre_t
                delta_value = value - pre_v
                # print(col_name, index, pre_t, th, pre_v ,value)
                rates.append(delta_value / delta_t)
                pre_t = th
                pre_v = value
            df[rate_col_name] = rates

        # 准备输出值 Y
        self.cct_names = []
        for x in self.df.columns:
            if "time" in x or "error" in x or "rate" in x:
                continue
            self.cct_names.append(x)
        self.rates_names = [f"{x}_rate" for x in self.cct_names]
        self.error_names = [f"{x}-error" for x in self.cct_names]

        self.cct = self.df[self.cct_names].values
        self.rates = self.df[self.rates_names].values
        self.errors = self.df[self.error_names].values
    
    
    def set_as_sim_dataset(self, dcdt_fuc, t_eval, y0, args):
        
        y = odeint(dcdt_fuc, y0=y0, t=t_eval, args=args)
        
        # y.shape (size, 10)
        df_new = pd.DataFrame(columns=['time'] + self.cct_names)
        df_new['time'] = t_eval
        
        for c_name, col_val in zip(self.cct_names, np.transpose(y, [1,0])):
            df_new[c_name] = col_val
            df_new[f"{c_name}-error"] = 0.001
        
        self.df = df_new
        self._setup()
    
    def set_dataset(self, c_pred_df):
        
        df = deepcopy(c_pred_df)
        df['time'] = self.df['time'].values
        for c_name in self.cct_names:
            c_error_name = f"{c_name}-error"
            n_len = len(df[c_name].values)
            _error = np.repeat(0.0001, n_len)
            df[c_error_name] = _error
        self.df = df
        self._setup()


    def get_rates(self):
        return self.rates

    def get_df(self):
        return self.df

    def get_errors(self):
        return self.errors

    def get_cct(self):
        return self.cct

    def get_var_col_names(self):
        return self.cct_names, self.rates_names, self.error_names

    def get_weights(self):
        max_value = self.df[self.cct_names].describe().loc['max'].values.max()

        vars_max = self.df[self.cct_names].describe().loc['max']
        weights = (max_value / vars_max).values

        return np.array(weights)

    def get_vars_max(self):
        vars_max = self.df[self.cct_names].describe().loc['max'].values
        return vars_max


def get_format_time(f_s=None):
    haomiao = str(time.time()).split('.')[-1]
    if f_s is None:
        f_s = "%Y-%-m-%d %H:%M:%S."
        return datetime.now().strftime(f_s) + haomiao
    return datetime.now().strftime(f_s)
def plot_dataset(dataset, dataset_pred=None):
    
    df = dataset.get_df()
    cct_names, rates_names, error_names = dataset.get_var_col_names()
    
    cols = 5
    rows = math.ceil(len(cct_names) / cols)

    fig, fig_axes = plt.subplots(ncols=cols, nrows=rows, figsize=(4.2 * cols, 4 * rows), dpi=100)
    if isinstance(fig_axes, np.ndarray):
        fig_axes = fig_axes.reshape(-1)
    else:
        fig_axes = [fig_axes]

    for i, axes in enumerate(fig_axes):
        if i >= len(cct_names):
            axes.axis('off')
            continue
        
        y_name = cct_names[i]
        Y = df[y_name].values
        axes.plot(df['time'].values, Y, 'g', label=f"ob")
        axes.set_ylabel(f'cct_{y_name}')
        axes.set_xlabel(f'time(h)')

        # axes.plot(df['time'].values, df[rates_names[i]].values, '+', label=f"rate")
        
        if dataset_pred:
            _df_pred = dataset_pred.get_df()
            t_eval = _df_pred['time'].values
            axes.plot(t_eval, _df_pred[y_name].values, 'r', label=f"c(t)")
        
        
        # axes.plot(t_eval, dcdt_df[y_name].values,'g', label=f"c'(t)")

        axes.legend()
        # axes.set_title(f"{y_name}", fontsize=14)

    plt.tight_layout()
    plt.show()

In [102]:
def get_dcdt_func(k_kinetics):
    k_kinetics = np.array(k_kinetics).astype(np.uint8)
    def _dcdt_func(t, c, p):
        r1 = p.k1 * c.xN2 if k_kinetics[0] == 1 else p.k1
        r2 = p.k2 * c.xNH3 if k_kinetics[1] == 1 else p.k2
        r3 = p.k3 * c.xNO2 if k_kinetics[2] == 1 else p.k3
        r4 = p.k4 * c.xNO3 if k_kinetics[3] == 1 else p.k4
        r5 = p.k5 * c.xNO2 if k_kinetics[4] == 1 else p.k5
        r6 = p.k6 * c.xNO2 * c.xNO3 if k_kinetics[5] == 1 else p.k6
        r7 = p.k7 * c.xNO3 if k_kinetics[6] == 1 else p.k7
        r8 = p.k8 * c.xNO3 if k_kinetics[7] == 1 else p.k8
        r9 = p.k9 * c.xNH3 if k_kinetics[8] == 1 else p.k9
        r10 = p.k10 * c.xNOrg if k_kinetics[9] == 1 else p.k10
        r11 = p.k11 * c.xNOrg if k_kinetics[10] == 1 else p.k11

        
        dc_xNH3 = 2 * r1 + r7 + r10 - r2 - r6 - r9
        dc_xNO3 = r3 - r7 - r4 - r8 + r11
        dc_xNO2 = r2 + r4 - r3 - r6 - 2 * r5
        dc_xNOrg = r8 + r9 - r10 - r11
        dc_xN2 = r5 + r6 - r1

        dc_ANH3 = (2 * r1 * (c.AN2 - c.ANH3) + (c.ANO3 - c.ANH3) * r7 + (c.ANOrg - c.ANH3) * r10) / c.xNH3
        dc_ANO3 = ((c.ANO2 - c.ANO3) * r2 + (c.ANOrg - c.ANO3) * r11) / c.xNO3
        dc_ANO2 = ((c.ANH3 - c.ANO2) * r2 + (c.ANO3 - c.ANO2) * r4) / c.xNO2
        dc_ANOrg = ((c.ANO3 - c.ANOrg) * r8 + (c.ANH3 - c.ANOrg) * r9) / c.xNOrg
        dc_AN2 = ((c.ANO2 - c.AN2) * r5 + (c.ANO2 * c.ANH3 - c.AN2) * r6) / c.xN2

        # dcdts = [dc_xNH3, dc_xNO3, dc_xNO2, dc_xNOrg, dc_xN2, dc_ANH3, dc_ANO3, dc_ANO2, dc_ANOrg, dc_AN2]
        
        dcdts =  {
            'xNH3': dc_xNH3,
            'xNO3': dc_xNO3,
            'xNO2': dc_xNO2,
            'xNOrg': dc_xNOrg,
            'xN2': dc_xN2,
            'ANH3': dc_ANH3,
            'ANO3': dc_ANO3,
            'ANO2': dc_ANO2,
            'ANOrg': dc_ANOrg,
            'AN2': dc_AN2,
        }
        return dcdts
    
    return _dcdt_func

In [103]:
def dcdt_func_for_odeint(c, t, ks, k_kinetics):
    
    # print(c, t, ks, k_kinetics)
    # print()
    c_xNH3, c_xNO3, c_xNO2, c_xNOrg, c_xN2, c_ANH3, c_ANO3, c_ANO2, c_ANOrg, c_AN2 = c
    
    r1 = ks[0] * c_xN2 if k_kinetics[0] == 1 else ks[0]
    r2 = ks[1] * c_xNH3 if k_kinetics[1] == 1 else ks[1]
    r3 = ks[2] * c_xNO2 if k_kinetics[2] == 1 else ks[2]
    r4 = ks[3] * c_xNO3 if k_kinetics[3] == 1 else ks[3]
    r5 = ks[4] * c_xNO2 if k_kinetics[4] == 1 else ks[4]
    r6 = ks[5] * c_xNO2 * c_xNO3 if k_kinetics[5] == 1 else ks[5]
    r7 = ks[6] * c_xNO3 if k_kinetics[6] == 1 else ks[6]
    r8 = ks[7] * c_xNO3 if k_kinetics[7] == 1 else ks[7]
    r9 = ks[8] * c_xNH3 if k_kinetics[8] == 1 else ks[8]
    r10 = ks[9] * c_xNOrg if k_kinetics[9] == 1 else ks[9]
    r11 = ks[10] * c_xNOrg if k_kinetics[10] == 1 else ks[10]
    
    dc_xNH3 = 2 * r1 + r7 + r10 - r2 - r6 - r9
    dc_xNO3 = r3 - r7 - r4 - r8 + r11
    dc_xNO2 = r2 + r4 - r3 - r6 - 2 * r5
    dc_xNOrg = r8 + r9 - r10 - r11
    dc_xN2 = r5 + r6 - r1
    dc_ANH3 = (2 * r1 * (c_AN2 - c_ANH3) + (c_ANO3 - c_ANH3) * r7 + (c_ANOrg - c_ANH3) * r10) / c_xNH3
    dc_ANO3 = ((c_ANO2 - c_ANO3) * r2 + (c_ANOrg - c_ANO3) * r11) / c_xNO3
    dc_ANO2 = ((c_ANH3 - c_ANO2) * r2 + (c_ANO3 - c_ANO2) * r4) / c_xNO2
    dc_ANOrg = ((c_ANO3 - c_ANOrg) * r8 + (c_ANH3 - c_ANOrg) * r9) / c_xNOrg
    dc_AN2 = ((c_ANO2 - c_AN2) * r5 + (c_ANO2 * c_ANH3 - c_AN2) * r6) / c_xN2

    dcdts = [dc_xNH3, dc_xNO3, dc_xNO2, dc_xNOrg, dc_xN2, dc_ANH3, dc_ANO3, dc_ANO2, dc_ANOrg, dc_AN2]

    return np.array(dcdts)

# simulator function
def competition_model(rng, t_eval, y0,  ks, k_kinetics, size=None):
    # print(y0)
    y = odeint(dcdt_func_for_odeint, y0=y0, t=t_eval, args=(ks, k_kinetics))
    return y


In [92]:
def r2_loss(pred, y):
    r2_loss = 1 - np.square(pred - y).sum() / np.square(y - np.mean(y)).sum()
    return r2_loss


def get_model(dataset, k_kinetics, k_sigma_priors=0.01, kf_type=0):

    df = dataset.get_df()
    times = df['time'].values
    
    errors = dataset.get_errors()
    rates = dataset.get_rates()
    cct_names, rates_names, error_names = dataset.get_var_col_names()
        
    # 定义参数优化模型
    mcmc_model = pm.Model()
    ## 参数个数
    params_n = 11
    parames ={}
    
    with mcmc_model:
        for ki in range(1, params_n + 1):
            if kf_type == 0:
                p_dense = pm.HalfNormal(f"k{ki}", sigma=k_sigma_priors)
            else:
                p_dense = pm.Normal(f"k{ki}",mu=0, sigma=k_sigma_priors)
            parames[f"k{ki}"] = (p_dense, ())
    
    # parames['k_kinetics'] = np.array(k_kinetics, dtype=np.float64)
    parames['extra']=  np.zeros(1)
    
    c0 = {}
    with mcmc_model:
        for c_name in cct_names:
            _maxx = df[c_name].values.max()
            c0[f"{c_name}"] = (pm.HalfNormal(f"{c_name}_s", sigma=_maxx), ())
            # c0[f"{c_name}"] = (pm.Normal(f"{c_name}_s", mu=_meanx, sigma=s), ())
        
    
        y_hat, _, problem, solver, _, _ = sunode.wrappers.as_pytensor.solve_ivp(
            y0=c0,
            params=parames,
            rhs=get_dcdt_func(k_kinetics),
            tvals=times,
            t0=times[0],
        )
        
        sd = pm.HalfNormal('sd')
        for c_name in cct_names:
            pm.Normal(f'{c_name}', mu=y_hat[f"{c_name}"], sigma=sd, observed=df[f"{c_name}"].values)
            pm.Deterministic(f'{c_name}_mu', y_hat[f"{c_name}"])

    return mcmc_model


In [93]:
def get_predict_ks(idata):
    parames_summary = az.summary(idata, round_to=10)
    ks_names = [f"k{x+1}" for x in range(11)]

    predict_ks = []
    for k_name in ks_names:
        k_v = parames_summary["mean"][k_name]
        predict_ks.append(k_v)
    return np.array(predict_ks)


In [94]:
# k_kinetics = np.repeat(1, 11).astype(np.uint8)
k_kinetics = np.array([0,1,1,1,1,0,0,1,1,1,0])
# ks = np.random.random(11)/100 # 先验k
ks = np.array([0.00071942, 0.00269696, 0.00498945, 0.00444931, 0.00571299, 0.00801272, 0.00131931, 0.00319959, 0.00415571, 0.00228432, 0.00177611])
#  =======================================================

# t_eval = np.arange(0, 150, 5)

dataset = MyDataset("dataset/data.csv")
df = dataset.get_df()
cct_names, rates_names, error_names = dataset.get_var_col_names()

# c0 = df[cct_names].iloc[0].values
# dataset.set_as_sim_dataset(dcdt_func_for_odeint, t_eval, c0, args=(ks, k_kinetics))
# df = dataset.get_df()

# plot_dataset(dataset=dataset)
mcmc_model = get_model(dataset, k_kinetics, k_sigma_priors=0.01, kf_type=0)


In [ ]:
plot_dataset(dataset=dataset)

In [ ]:
idata = pm.sample(draws=1000, model=mcmc_model, chains=4, cores=4)

In [104]:
def get_result(dataset, idata):
    res_df = az.summary(idata)
    
    predict_ks = get_predict_ks(idata=idata)
    

    df = dataset.get_df()
    times = df['time'].values
    
    errors = dataset.get_errors()
    rates = dataset.get_rates()
    cct_names, rates_names, error_names = dataset.get_var_col_names()
    
    c_df = pd.DataFrame(columns=cct_names)
    t_n = len(times)
    for c_name in cct_names:
        c_value = []
        for i in range(t_n):
            _x = res_df['mean'][f"{c_name}_mu[{i}]"]
            c_value.append(_x)
        c_df[c_name] = c_value
    
    return predict_ks, c_df


predict_ks, c_df = get_result(dataset, idata)

print(predict_ks)
print(c_df[cct_names])
# print(az.summary(idata))

array([9.46627500e-04, 1.80282300e-04, 7.75364680e-03, 2.25054380e-03,
       9.93785290e-03, 1.11419580e-03, 2.06023280e-03, 1.56184350e-03,
       1.66389505e-02, 6.79560000e-05, 1.19875624e-02])

,xNH3,xNO3,xNO2,xNOrg,xN2,ANH3,ANO3,ANO2,ANOrg,AN2
0,0.924,2.851,0.069,62.093,0.000,8.082,0.621,0.060,0.059,0.026
1,0.649,2.862,0.155,62.135,0.068,4.992,0.548,0.320,0.124,0.651
2,0.538,2.903,0.173,62.011,0.156,2.712,0.498,0.321,0.151,0.534
3,0.492,2.950,0.178,61.824,0.246,1.413,0.460,0.330,0.162,0.456


In [ ]:
c_df['time']  = df['time'].values
dataset_new = MyDataset("dataset/data.csv")
dataset_new.set_dataset(c_df)

plot_dataset(dataset, dataset_pred=dataset_new)


In [106]:
r2_all = r2_loss(dataset.get_cct(), c_df[cct_names].values)
print(r2_all)

for c_name in cct_names:
    _loss = r2_loss(dataset.df[c_name].values, c_df[c_name].values)
    print(f"r2: {c_name} {_loss}")


0.9992822993532159

'r2: xNH3 -0.8926770700262396'

'r2: xNO3 -85.8514520337752'

'r2: xNO2 -5.94321655742469'

'r2: xNOrg -13.762082355263884'

...

'r2: ANH3 0.9865236241877875'

'r2: ANO3 -448.7425181981374'

'r2: ANO2 -2.4792918569177638'

'r2: ANOrg -3.5620796821307694'

'r2: AN2 -2.8811935399551487'

In [ ]:
k_kinetics = np.array([0,1,1,1,1,0,0,1,1,1,0])

ks = predict_ks
t_eval = np.arange(0, 150, 1)
c0 = df[cct_names].iloc[0].values
cs = odeint(dcdt_func_for_odeint, c0, t=t_eval, args=(ks, k_kinetics))
# res = odeint(func=dcdt_func2, y0=c0, t=t_eval, args=(ks,k_kinetics))
# cs = np.transpose(res, [1,0])

print(cs.shape)

cols = 5
rows = math.ceil(len(cct_names) / cols)

fig, fig_axes = plt.subplots(ncols=cols, nrows=rows, figsize=(4.2 * cols, 4 * rows), dpi=100)
if isinstance(fig_axes, np.ndarray):
    fig_axes = fig_axes.reshape(-1)
else:
    fig_axes = [fig_axes]

for i, axes in enumerate(fig_axes):
    if i >= len(cct_names):
        axes.axis('off')
        continue

    y_name = cct_names[i]
    Y = df[y_name].values
    axes.plot(df['time'].values, Y, '*', label=f"ob")
    axes.set_ylabel(f'cct_{y_name}')
    axes.set_xlabel(f'time(h)')

    # axes.plot(df['time'].values, df[rates_names[i]].values, '+', label=f"rate")

    axes.plot(t_eval, cs[:, i], 'r', label=f"c(t)")
    # axes.plot(t_eval, dcdt_df[y_name].values,'g', label=f"c'(t)")

    axes.legend()
    # axes.set_title(f"{y_name}", fontsize=14)

plt.tight_layout()
plt.show()



In [63]:
import pickle
pickle.dump(idata,open("abc.np", 'wb'))


In [64]:
aa = pickle.load(open("abc.np", 'rb'))
az.summary(aa)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
k1,0.008,0.006,0.000,0.019,0.000,0.000,1123.0,969.0,1.01
k2,0.000,0.000,0.000,0.000,0.000,0.000,340.0,1210.0,1.02
k3,0.008,0.006,0.000,0.019,0.000,0.000,678.0,714.0,1.01
k4,0.001,0.002,0.000,0.005,0.000,0.000,138.0,1686.0,1.05
k5,0.009,0.006,0.000,0.020,0.000,0.000,487.0,990.0,1.01
...,...,...,...,...,...,...,...,...,...
ANOrg_mu[3],0.170,0.062,0.059,0.285,0.002,0.001,1026.0,1272.0,1.00
AN2_mu[0],0.024,0.018,0.000,0.058,0.000,0.000,1817.0,1145.0,1.00
AN2_mu[1],0.495,0.324,0.039,1.075,0.025,0.018,147.0,1687.0,1.04
AN2_mu[2],0.454,0.225,0.085,0.881,0.010,0.007,537.0,2193.0,1.02


In [65]:
kk_list_all = list(product([0,1], repeat=11))

In [76]:
a = kk_list_all[2]
x = "".join([f"{x}" for x in a])
int(x,2)

2

In [83]:
a = bin(2)[2:]
f"{a:0>11}"

'00000000010'

In [88]:
def get_dcdt(a):

    def _re(b):
        print(a,b)
    return _re

fun1  = get_dcdt(1)
fun2 = get_dcdt(2)

fun1("func1")
fun2("func12")

1

'func1'

2

'func12'